In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from operator import itemgetter

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [11]:
path = './data/'
Train_data = pd.read_csv(path+'used_car_train_20200313.csv', sep=' ')
Test_data = pd.read_csv(path+'used_car_testB_20200421.csv', sep=' ')
print(Train_data.shape)
print(Test_data.shape)

(150000, 31)
(50000, 30)


In [12]:
print(Train_data.columns)
# print(Train_data.info())

Index(['SaleID', 'name', 'regDate', 'model', 'brand', 'bodyType', 'fuelType',
       'gearbox', 'power', 'kilometer', 'notRepairedDamage', 'regionCode',
       'seller', 'offerType', 'creatDate', 'price', 'v_0', 'v_1', 'v_2', 'v_3',
       'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12',
       'v_13', 'v_14'],
      dtype='object')


In [13]:
numerical_features = ['power', 'kilometer', 'v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13','v_14' ]

categorical_features = ['name', 'model', 'brand', 'bodyType', 'fuelType', 'gearbox', 'notRepairedDamage', 'regionCode']

In [14]:
# 处理异常值
# 使用Tukeys tets方法
def handle_outliers(df, col_name, scale=3):
    def tukeys_test(df, col_name, scale):
        iqr = scale*(df[col_name].quantile(0.75) - df[col_name].quantile(0.25))
        min_val = df[col_name].quantile(0.25) - iqr
        max_val = df[col_name].quantile(0.75) + iqr
          
        return min_val, max_val
    
    
    
    min_val, max_val = tukeys_test(df, col_name, scale)

    del_idx = df[(df[col_name] > max_val) | (df[col_name]<min_val) ].index
    
    
    print("# of delete rows: {}".format(len(del_idx)))
    
    
    Train_data_n = df.drop(del_idx)
    print('# of current rows: {}'.format(len(df)))
    
    return Train_data_n

    



for fea in numerical_features:
    print('Handling {}'.format(fea))
    Train_data_n= handle_outliers(Train_data, 'power')

Handling power
# of delete rows: 963
# of current rows: 150000
Handling kilometer
# of delete rows: 963
# of current rows: 150000
Handling v_0
# of delete rows: 963
# of current rows: 150000
Handling v_1
# of delete rows: 963
# of current rows: 150000
Handling v_2
# of delete rows: 963
# of current rows: 150000
Handling v_3
# of delete rows: 963
# of current rows: 150000
Handling v_4
# of delete rows: 963
# of current rows: 150000
Handling v_5
# of delete rows: 963
# of current rows: 150000
Handling v_6
# of delete rows: 963
# of current rows: 150000
Handling v_7
# of delete rows: 963
# of current rows: 150000
Handling v_8
# of delete rows: 963
# of current rows: 150000
Handling v_9
# of delete rows: 963
# of current rows: 150000
Handling v_10
# of delete rows: 963
# of current rows: 150000
Handling v_11
# of delete rows: 963
# of current rows: 150000
Handling v_12
# of delete rows: 963
# of current rows: 150000
Handling v_13
# of delete rows: 963
# of current rows: 150000
Handling v_1

In [15]:
# print(len(Train_data))
# print(len(Train_data_n))

In [16]:
# 特征构造

In [17]:
Train_data_n['train']=1
Test_data['train']=0
merged_data = pd.concat([Train_data_n, Test_data], ignore_index=True)

In [18]:
print(merged_data.shape)

(199037, 32)


In [19]:
# 创建新特征，出售时间-购买时间
print(merged_data['regDate'].head())
print(merged_data['creatDate'].head())


0    20040402
1    20030301
2    20040403
3    19960908
4    20120103
Name: regDate, dtype: int64
0    20160404
1    20160309
2    20160402
3    20160312
4    20160313
Name: creatDate, dtype: int64


In [20]:
merged_data['used_days'] = (pd.to_datetime(merged_data['creatDate'], format='%Y%m%d', errors='coerce')
- pd.to_datetime(merged_data['regDate'], format='%Y%m%d', errors='coerce')).dt.days

In [21]:
merged_data['used_year'] = round(merged_data['used_days'] / 365, 1)#二手车使用年数

In [22]:
print('{} missing values in \'used_days\' feature'.format(merged_data['used_days'].isnull().sum()))

15054 missing values in 'used_days' feature


In [23]:
# 邮编提取关键字
merged_data['regionCode'].head()

0    1046
1    4366
2    2806
3     434
4    6977
Name: regionCode, dtype: int64

In [24]:
# 尝试从邮编中提取城市信息
merged_data['city'] = merged_data['regionCode'].apply(lambda x : str(x)[:-3])
merged_data = merged_data

In [25]:
merged_data.columns

Index(['SaleID', 'name', 'regDate', 'model', 'brand', 'bodyType', 'fuelType',
       'gearbox', 'power', 'kilometer', 'notRepairedDamage', 'regionCode',
       'seller', 'offerType', 'creatDate', 'price', 'v_0', 'v_1', 'v_2', 'v_3',
       'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12',
       'v_13', 'v_14', 'train', 'used_days', 'used_year', 'city'],
      dtype='object')

In [26]:
def feat_price_cal(feat_list,dataset):
    
    for feat_name in feat_list:
        df_grouby = dataset.groupby(feat_name)
        all_info={}
        for feat_value, item in df_grouby:
            info = {}
            curr_data = item[item['price']>0]
            info[feat_name+'_amount'] = len(curr_data)
            info[feat_name+'_price_avg'] = curr_data['price'].mean()
            info[feat_name+'_price_median'] = curr_data['price'].median()
            info[feat_name+'_price_min'] = curr_data['price'].min()
            info[feat_name+'_price_max'] = curr_data['price'].max()
            info[feat_name+'_price_sum'] = curr_data['price'].sum()
            info[feat_name+'_price_std'] = curr_data['price'].std()
#             print(info)
            all_info[feat_value] = info
        feat_info_df = pd.DataFrame(all_info).T.reset_index().rename(columns={"index": feat_name})
#         print(feat_info_df.T.head())
        dataset = dataset.merge(feat_info_df, how='left', on=feat_name)
    return dataset

In [27]:
gb_feats = ['brand','model','kilometer','fuelType','bodyType']
dataset = feat_price_cal(gb_feats, merged_data)
print(dataset.columns)

Index(['SaleID', 'name', 'regDate', 'model', 'brand', 'bodyType', 'fuelType',
       'gearbox', 'power', 'kilometer', 'notRepairedDamage', 'regionCode',
       'seller', 'offerType', 'creatDate', 'price', 'v_0', 'v_1', 'v_2', 'v_3',
       'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12',
       'v_13', 'v_14', 'train', 'used_days', 'used_year', 'city',
       'brand_amount', 'brand_price_avg', 'brand_price_median',
       'brand_price_min', 'brand_price_max', 'brand_price_sum',
       'brand_price_std', 'model_amount', 'model_price_avg',
       'model_price_median', 'model_price_min', 'model_price_max',
       'model_price_sum', 'model_price_std', 'kilometer_amount',
       'kilometer_price_avg', 'kilometer_price_median', 'kilometer_price_min',
       'kilometer_price_max', 'kilometer_price_sum', 'kilometer_price_std',
       'fuelType_amount', 'fuelType_price_avg', 'fuelType_price_median',
       'fuelType_price_min', 'fuelType_price_max', 'fuelType_price_sum',
      

In [28]:
merged_data = dataset

In [29]:
merged_data.columns

Index(['SaleID', 'name', 'regDate', 'model', 'brand', 'bodyType', 'fuelType',
       'gearbox', 'power', 'kilometer', 'notRepairedDamage', 'regionCode',
       'seller', 'offerType', 'creatDate', 'price', 'v_0', 'v_1', 'v_2', 'v_3',
       'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12',
       'v_13', 'v_14', 'train', 'used_days', 'used_year', 'city',
       'brand_amount', 'brand_price_avg', 'brand_price_median',
       'brand_price_min', 'brand_price_max', 'brand_price_sum',
       'brand_price_std', 'model_amount', 'model_price_avg',
       'model_price_median', 'model_price_min', 'model_price_max',
       'model_price_sum', 'model_price_std', 'kilometer_amount',
       'kilometer_price_avg', 'kilometer_price_median', 'kilometer_price_min',
       'kilometer_price_max', 'kilometer_price_sum', 'kilometer_price_std',
       'fuelType_amount', 'fuelType_price_avg', 'fuelType_price_median',
       'fuelType_price_min', 'fuelType_price_max', 'fuelType_price_sum',
      

In [30]:
# 行程与功率关联统计
kk = ['kilometer','power']
t1 = Train_data_n.groupby(kk[0],as_index=False)[kk[1]].agg(
        {kk[0]+'_'+kk[1]+'_count':'count',kk[0]+'_'+kk[1]+'_max':'max',kk[0]+'_'+kk[1]+'_median':'median',
         kk[0]+'_'+kk[1]+'_min':'min',kk[0]+'_'+kk[1]+'_sum':'sum',kk[0]+'_'+kk[1]+'_std':'std',kk[0]+'_'+kk[1]+'_mean':'mean'})
merged_data = pd.merge(merged_data,t1,on=kk[0],how='left')

In [31]:
merged_data.columns

Index(['SaleID', 'name', 'regDate', 'model', 'brand', 'bodyType', 'fuelType',
       'gearbox', 'power', 'kilometer', 'notRepairedDamage', 'regionCode',
       'seller', 'offerType', 'creatDate', 'price', 'v_0', 'v_1', 'v_2', 'v_3',
       'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12',
       'v_13', 'v_14', 'train', 'used_days', 'used_year', 'city',
       'brand_amount', 'brand_price_avg', 'brand_price_median',
       'brand_price_min', 'brand_price_max', 'brand_price_sum',
       'brand_price_std', 'model_amount', 'model_price_avg',
       'model_price_median', 'model_price_min', 'model_price_max',
       'model_price_sum', 'model_price_std', 'kilometer_amount',
       'kilometer_price_avg', 'kilometer_price_median', 'kilometer_price_min',
       'kilometer_price_max', 'kilometer_price_sum', 'kilometer_price_std',
       'fuelType_amount', 'fuelType_price_avg', 'fuelType_price_median',
       'fuelType_price_min', 'fuelType_price_max', 'fuelType_price_sum',
      

In [32]:
# # 对属性作统计
# # 只使用训练集的做统计，以防止数据泄露

# Train_data_n_gb = Train_data_n.groupby('brand')
# all_info={}
# for brand_name, name_data in Train_data_n_gb:
#     info={}
#     curr_data = name_data[name_data['price']>0]
#     info['brand_amount'] = len(curr_data)
#     info['brand_price_avg'] = curr_data['price'].mean()
#     info['brand_price_median'] = curr_data['price'].median()
#     info['brand_price_min'] = curr_data['price'].min()
#     info['brand_price_max'] = curr_data['price'].max()
#     info['brand_price_sum'] = curr_data['price'].sum()
#     info['brand_price_std'] = curr_data['price'].std()
#     all_info[brand_name] = info
#     print(pd.DataFrame(all_info).T.head())
# brand_info_df = pd.DataFrame(all_info).T.reset_index().rename(columns={"index": "brand"})
# merged_data = merged_data.merge(brand_info_df, how='left', on='brand')


In [33]:
# # 对model属性作统计
# Train_data_n_gb = Train_data_n.groupby('model')
# all_info={}
# for brand_name, name_data in Train_data_n_gb:
#     info={}
#     curr_data = name_data[name_data['price']>0]
#     info['model_amount'] = len(curr_data)
#     info['model_price_avg'] = curr_data['price'].mean()
#     info['model_price_median'] = curr_data['price'].median()
#     info['model_price_min'] = curr_data['price'].min()
#     info['model_price_max'] = curr_data['price'].max()
#     info['model_price_sum'] = curr_data['price'].sum()
#     info['model_price_std'] = curr_data['price'].std()
#     all_info[brand_name] = info
# brand_info_df = pd.DataFrame(all_info).T.reset_index().rename(columns={"index": "model"})
# merged_data = merged_data.merge(brand_info_df, how='left', on='model')


In [34]:
# # gear_box做统计
# cat_cols = ['kilometer','fuelType','bodyType','gearbox']
# for col in cat_cols:
#     t = Train_data_n.groupby(col,as_index=False)['price'].agg(
#         {col+'_count':'count',col+'_price_max':'max',col+'_price_median':'median',
#          col+'_price_min':'min',col+'_price_sum':'sum',col+'_price_std':'std',col+'_price_mean':'mean'})
#     merged_data = pd.merge(merged_data,t,on=col,how='left')

In [35]:
# len(merged_data.columns)

In [36]:
# 对power进行分桶,对训练集和测试集都实现
bin = [i*10 for i in range(31)]
merged_data['power_bin'] = pd.cut(Train_data_n['power'], bin, labels=False)

# 尝试对model进行分桶
bin = [i*10 for i in range(24)]
merged_data['model_bin'] = pd.cut(Train_data_n['model'], bin, labels=False)

In [37]:
merged_data.columns

Index(['SaleID', 'name', 'regDate', 'model', 'brand', 'bodyType', 'fuelType',
       'gearbox', 'power', 'kilometer', 'notRepairedDamage', 'regionCode',
       'seller', 'offerType', 'creatDate', 'price', 'v_0', 'v_1', 'v_2', 'v_3',
       'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12',
       'v_13', 'v_14', 'train', 'used_days', 'used_year', 'city',
       'brand_amount', 'brand_price_avg', 'brand_price_median',
       'brand_price_min', 'brand_price_max', 'brand_price_sum',
       'brand_price_std', 'model_amount', 'model_price_avg',
       'model_price_median', 'model_price_min', 'model_price_max',
       'model_price_sum', 'model_price_std', 'kilometer_amount',
       'kilometer_price_avg', 'kilometer_price_median', 'kilometer_price_min',
       'kilometer_price_max', 'kilometer_price_sum', 'kilometer_price_std',
       'fuelType_amount', 'fuelType_price_avg', 'fuelType_price_median',
       'fuelType_price_min', 'fuelType_price_max', 'fuelType_price_sum',
      

In [38]:
# 处理repairedDamage的‘-’值
def replaceNotRepairedDamage(df):
    df['notRepairedDamage'].replace('-', np.nan, inplace=True)
replaceNotRepairedDamage(merged_data)

In [39]:
merged_data['notRepairedDamage'] = merged_data[['notRepairedDamage']].astype('float64')

In [40]:
merged_data['city'] = pd.to_numeric(merged_data['city'], errors = 'coerce')

In [41]:
# 统计各种name的数量
merged_data['name_count'] = merged_data.groupby(['name'])['SaleID'].transform('count')



In [42]:
# 类别型变量独热编码
cate_oht_features = ['bodyType', 'fuelType','gearbox', 'notRepairedDamage', 'power_bin','city']  
merged_data = pd.get_dummies(merged_data, columns=cate_oht_features)
merged_data['model'] = merged_data['model'].fillna(0)

In [43]:
# 对price做Log处理
merged_data['price'] = np.log1p(merged_data['price'])

In [44]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199037 entries, 0 to 199036
Columns: 131 entries, SaleID to city_8.0
dtypes: float64(58), int64(16), uint8(57)
memory usage: 124.7 MB


In [45]:
anon_feat = ['v_0', 'v_3', 'v_8', 'v_12']
anon_df = merged_data[anon_feat]
anon_df.head()

,v_0,v_3,v_8,v_12
0,43.357796,2.159744,0.022816,-2.420821
1,45.305273,1.380657,0.026597,-1.030483
2,45.978359,-0.998467,0.062173,1.565330
3,45.687478,0.883600,0.033395,-0.501868
4,44.383511,-1.571239,0.078819,0.931110


In [46]:
# 线性组合
num_cols = [0,3,8,12]
for i in num_cols:
    for j in num_cols:
        merged_data['new'+str(i)+'+'+str(j)]=merged_data['v_'+str(i)]+merged_data['v_'+str(j)]


for i in num_cols:
    for j in num_cols:
        merged_data['new'+str(i)+'-'+str(j)]=merged_data['v_'+str(i)]-merged_data['v_'+str(j)]


for i in range(15):
    merged_data['new'+str(i)+'*year']=merged_data['v_'+str(i)] * merged_data['used_year']

In [47]:
# 非线性组合

In [48]:
from sklearn.preprocessing import PolynomialFeatures
poly_scaler = PolynomialFeatures(4, interaction_only=True)
mul_feat = poly_scaler.fit_transform(anon_df)

In [49]:
feat_name = poly_scaler.get_feature_names(anon_feat)
print(feat_name)

['1', 'v_0', 'v_3', 'v_8', 'v_12', 'v_0 v_3', 'v_0 v_8', 'v_0 v_12', 'v_3 v_8', 'v_3 v_12', 'v_8 v_12', 'v_0 v_3 v_8', 'v_0 v_3 v_12', 'v_0 v_8 v_12', 'v_3 v_8 v_12', 'v_0 v_3 v_8 v_12']


In [50]:
print(mul_feat[0][1:5])

[ 4.33577963e+01  2.15974409e+00  2.28163674e-02 -2.42082079e+00]


In [51]:
info = {}
for ii, name in enumerate(feat_name[5:]):
    print(name)
    info['poly_feat_' + name.strip()] = mul_feat[:,ii+5]

mul_feat_df = pd.DataFrame(info)

v_0 v_3
v_0 v_8
v_0 v_12
v_3 v_8
v_3 v_12
v_8 v_12
v_0 v_3 v_8
v_0 v_3 v_12
v_0 v_8 v_12
v_3 v_8 v_12
v_0 v_3 v_8 v_12


In [52]:
mul_feat_df.head()
print(mul_feat_df.shape)
print(merged_data.shape)

(199037, 11)
(199037, 178)


In [53]:
pd.concat([mul_feat_df, merged_data['price']], axis=1).corr()['price']

poly_feat_v_0 v_3            -0.921635
poly_feat_v_0 v_8             0.859377
poly_feat_v_0 v_12            0.832402
poly_feat_v_3 v_8            -0.785649
poly_feat_v_3 v_12            0.011602
poly_feat_v_8 v_12            0.673454
poly_feat_v_0 v_3 v_8        -0.778799
poly_feat_v_0 v_3 v_12       -0.037967
poly_feat_v_0 v_8 v_12        0.702136
poly_feat_v_3 v_8 v_12       -0.527982
poly_feat_v_0 v_3 v_8 v_12   -0.537857
price                         1.000000
Name: price, dtype: float64

In [54]:
print(merged_data.shape)
merged_data = pd.concat([merged_data, mul_feat_df], axis=1)
print(merged_data.shape)


(199037, 178)
(199037, 189)


In [55]:
# polymerged_data_polyfeat.filter(regex=("poly.*"))

In [ ]:
merged_data.corr()['price'].to_csv('./data/corr.csv')

In [57]:
merged_data.columns

Index(['SaleID', 'name', 'regDate', 'model', 'brand', 'power', 'kilometer',
       'regionCode', 'seller', 'offerType',
       ...
       'poly_feat_v_0 v_8', 'poly_feat_v_0 v_12', 'poly_feat_v_3 v_8',
       'poly_feat_v_3 v_12', 'poly_feat_v_8 v_12', 'poly_feat_v_0 v_3 v_8',
       'poly_feat_v_0 v_3 v_12', 'poly_feat_v_0 v_8 v_12',
       'poly_feat_v_3 v_8 v_12', 'poly_feat_v_0 v_3 v_8 v_12'],
      dtype='object', length=189)

In [59]:
merged_data.shape

(199037, 189)

In [60]:
# 保存数据给树模型用
merged_data.to_csv('./data/merged_data_for_tree.gz',compression='gzip', index=0)

<h2>继续构造特征给LR或NN使用<h2>

In [ ]:
# 对特征进行归一化处理
from sklearn.preprocessing import StandardScaler



curr_scaler = StandardScaler()
curr_scaler.fit(merged_data[merged_data['train']==1]['power'].values.reshape(-1, 1))
curr_scaler.transform(merged_data['power'])



# for feat in numerical_features:
#     curr_scaler = StandardScaler()
#     # reshape成2d array
#     curr_scaler.fit(merged_data[merged_data['train']==1][feat].values.reshape(-1, 1))
#     curr_scaler.transform

In [ ]:
# 处理缺失值

In [ ]:
Train_data['train']=1
Test_data['train']=0
data = pd.concat([Train_data, Test_data], ignore_index=True)
print(data.columns)

In [ ]:
print('bodyType\n',Train_data_n['bodyType'].value_counts())
print('fuelType\n',Train_data_n['fuelType'].value_counts())
print('gearbox\n',Train_data_n['gearbox'].value_counts())
print('notRepairedDamage\n',Train_data_n['notRepairedDamage'].value_counts())
print('notRepairedDamage\n',Test_data['notRepairedDamage'].value_counts())

In [ ]:
Train_data_n.isnull().sum()

In [ ]:
Test_data[Test_data['bodyType'].isnull()].head()

In [ ]:
# 处理notRepairedDamage的‘-’缺失值
def replaceNotRepairedDamage(df):
    df['notRepairedDamage'].replace('-', np.nan, inplace=True)
    
replaceNotRepairedDamage(Train_data_n)
replaceNotRepairedDamage(Test_data)

In [ ]:
print(Train_data_n['notRepairedDamage'].value_counts())

In [ ]:
print(Test_data['notRepairedDamage'].value_counts())

In [ ]:
# 用众数填充gearbox和fueltype属性
from sklearn.impute import SimpleImputer
imp_mf = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp_mf.fit(Test_data)

# 用训练集fit，用训练的scaler填充训练集和测试集
imp_mf.transform(Train_data_n)
imp_mf.transform(Test_data)